In [1]:
!ls ../01-intro/data

green_tripdata_2021-01.parquet	 yellow_tripdata_2022-01.parquet
green_tripdata_2021-02.parquet	 yellow_tripdata_2022-02.parquet
yellow_tripdata_2021-01.parquet


In [2]:
!pwd

/home/ubuntu/mlops-zoomcamp/02-experiment-tracking


In [3]:
!python -V

Python 3.10.9


In [4]:
!whoami

ubuntu


In [5]:
import pandas as pd

In [6]:
import pickle

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [9]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
try:
    experiment_id = mlflow.get_experiment_by_name("nyc-taxi-experiment").experiment_id
except AttributeError:
    experiment_id = mlflow.create_experiment("nyc-taxi-experiment")

In [10]:
experiment_id

'1'

In [11]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
import os
print(os.getcwd())

/home/ubuntu/mlops-zoomcamp/02-experiment-tracking


In [13]:
df_train = read_dataframe('../01-intro/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../01-intro/data/green_tripdata_2021-02.parquet')

In [14]:
len(df_train), len(df_val)

(73908, 61921)

In [15]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [16]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715207983167

In [19]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [21]:
with mlflow.start_run(experiment_id=experiment_id):

    mlflow.set_tag("developer", "ahmad")

    mlflow.log_param("train-data-path", "../01-intro/data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "../01-intro/data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [22]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run(experiment_id=experiment_id):
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[12:36:31] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                                                      
[0]	validation-rmse:19.64506                                                                                                                                                                
[1]	validation-rmse:18.24935                                                                                                                                                                
[2]	validation-rmse:16.98528                                                                                                                                                                
[3]	validation-rmse:15.84911                                                                                                                                                                
[4]	validation-rmse:14.81608                           

[42]	validation-rmse:6.69546                                                                                                                                                                
[43]	validation-rmse:6.68546                                                                                                                                                                
[44]	validation-rmse:6.67656                                                                                                                                                                
[45]	validation-rmse:6.66838                                                                                                                                                                
[46]	validation-rmse:6.65999                                                                                                                                                                
[47]	validation-rmse:6.65299                           

[85]	validation-rmse:6.57112                                                                                                                                                                
[86]	validation-rmse:6.57033                                                                                                                                                                
[87]	validation-rmse:6.56978                                                                                                                                                                
[88]	validation-rmse:6.56921                                                                                                                                                                
[89]	validation-rmse:6.56854                                                                                                                                                                
[90]	validation-rmse:6.56827                           

[128]	validation-rmse:6.55327                                                                                                                                                               
[129]	validation-rmse:6.55328                                                                                                                                                               
[130]	validation-rmse:6.55303                                                                                                                                                               
[131]	validation-rmse:6.55264                                                                                                                                                               
[132]	validation-rmse:6.55192                                                                                                                                                               
[133]	validation-rmse:6.55150                          

[171]	validation-rmse:6.54279                                                                                                                                                               
[172]	validation-rmse:6.54256                                                                                                                                                               
[173]	validation-rmse:6.54252                                                                                                                                                               
[174]	validation-rmse:6.54281                                                                                                                                                               
[175]	validation-rmse:6.54189                                                                                                                                                               
[176]	validation-rmse:6.54179                          

...

[669]	validation-rmse:6.33754                                                                                                                                                               
[670]	validation-rmse:6.33751                                                                                                                                                               
[671]	validation-rmse:6.33750                                                                                                                                                               
[672]	validation-rmse:6.33729                                                                                                                                                               
[673]	validation-rmse:6.33707                                                                                                                                                               
[674]	validation-rmse:6.33713                          

[712]	validation-rmse:6.33567                                                                                                                                                               
[713]	validation-rmse:6.33558                                                                                                                                                               
[714]	validation-rmse:6.33558                                                                                                                                                               
[715]	validation-rmse:6.33564                                                                                                                                                               
[716]	validation-rmse:6.33563                                                                                                                                                               
[717]	validation-rmse:6.33568                          

[755]	validation-rmse:6.33612                                                                                                                                                               
[756]	validation-rmse:6.33612                                                                                                                                                               
[757]	validation-rmse:6.33613                                                                                                                                                               
[758]	validation-rmse:6.33616                                                                                                                                                               
[759]	validation-rmse:6.33606                                                                                                                                                               
[760]	validation-rmse:6.33597                          

[33]	validation-rmse:6.48843                                                                                                                                                                
[34]	validation-rmse:6.48730                                                                                                                                                                
[35]	validation-rmse:6.48733                                                                                                                                                                
[36]	validation-rmse:6.48628                                                                                                                                                                
[37]	validation-rmse:6.48408                                                                                                                                                                
[38]	validation-rmse:6.48380                           

KeyboardInterrupt: 

In [33]:
mlflow.xgboost.autolog(disable=True)

In [34]:
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1685109896507, experiment_id='1', last_update_time=1685109896507, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [35]:
with mlflow.start_run(experiment_id=experiment_id):
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[13:53:41] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95635
[2]	validation-rmse:16.59103
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29029
[5]	validation-rmse:13.32807
[6]	validation-rmse:12.47571
[7]	validation-rmse:11.72138
[8]	validation-rmse:11.05825
[9]	validation-rmse:10.47535
[10]	validation-rmse:9.96449
[11]	validation-rmse:9.51756
[12]	validation-rmse:9.12641
[13]	validation-rmse:8.78569
[14]	validation-rmse:8.49089
[15]	validation-rmse:8.23445
[16]	validation-rmse:8.01207
[17]	validation-rmse:7.81886
[18]	validation-rmse:7.65169
[19]	validation-rmse:7.50672
[20]	validation-rmse:7.38153
[21]	validation-rmse:7.27299
[22]	validation-rmse:7.17863
[23]	validation-rmse:7.09718
[24]	validation-rmse:7.02603
[25]	validation-rmse:6.96344
[26]	validation-rmse:6.90935
[27]	validation-rmse:6.86285
[28]	validation-rmse:6.82157
[29]	validation-rmse:6.78486
[30]	valida

[273]	validation-rmse:6.38760
[274]	validation-rmse:6.38752
[275]	validation-rmse:6.38711
[276]	validation-rmse:6.38693
[277]	validation-rmse:6.38679
[278]	validation-rmse:6.38631
[279]	validation-rmse:6.38589
[280]	validation-rmse:6.38573
[281]	validation-rmse:6.38517
[282]	validation-rmse:6.38490
[283]	validation-rmse:6.38479
[284]	validation-rmse:6.38458
[285]	validation-rmse:6.38413
[286]	validation-rmse:6.38405
[287]	validation-rmse:6.38383
[288]	validation-rmse:6.38357
[289]	validation-rmse:6.38302
[290]	validation-rmse:6.38281
[291]	validation-rmse:6.38251
[292]	validation-rmse:6.38235
[293]	validation-rmse:6.38202
[294]	validation-rmse:6.38163
[295]	validation-rmse:6.38126
[296]	validation-rmse:6.38063
[297]	validation-rmse:6.38038
[298]	validation-rmse:6.37990
[299]	validation-rmse:6.37973
[300]	validation-rmse:6.37959
[301]	validation-rmse:6.37926
[302]	validation-rmse:6.37923
[303]	validation-rmse:6.37911
[304]	validation-rmse:6.37905
[305]	validation-rmse:6.37882
[306]	vali

[547]	validation-rmse:6.33673
[548]	validation-rmse:6.33664
[549]	validation-rmse:6.33674
[550]	validation-rmse:6.33657
[551]	validation-rmse:6.33638
[552]	validation-rmse:6.33626
[553]	validation-rmse:6.33601
[554]	validation-rmse:6.33603
[555]	validation-rmse:6.33590
[556]	validation-rmse:6.33570
[557]	validation-rmse:6.33575
[558]	validation-rmse:6.33562
[559]	validation-rmse:6.33544
[560]	validation-rmse:6.33541
[561]	validation-rmse:6.33536
[562]	validation-rmse:6.33521
[563]	validation-rmse:6.33495
[564]	validation-rmse:6.33483
[565]	validation-rmse:6.33465
[566]	validation-rmse:6.33451
[567]	validation-rmse:6.33442
[568]	validation-rmse:6.33436
[569]	validation-rmse:6.33431
[570]	validation-rmse:6.33419
[571]	validation-rmse:6.33422
[572]	validation-rmse:6.33411
[573]	validation-rmse:6.33407
[574]	validation-rmse:6.33394
[575]	validation-rmse:6.33374
[576]	validation-rmse:6.33359
[577]	validation-rmse:6.33329
[578]	validation-rmse:6.33320
[579]	validation-rmse:6.33304
[580]	vali

[821]	validation-rmse:6.31206
[822]	validation-rmse:6.31200
[823]	validation-rmse:6.31205
[824]	validation-rmse:6.31194
[825]	validation-rmse:6.31188
[826]	validation-rmse:6.31180
[827]	validation-rmse:6.31174
[828]	validation-rmse:6.31150
[829]	validation-rmse:6.31147
[830]	validation-rmse:6.31113
[831]	validation-rmse:6.31115
[832]	validation-rmse:6.31116
[833]	validation-rmse:6.31122
[834]	validation-rmse:6.31111
[835]	validation-rmse:6.31107
[836]	validation-rmse:6.31105
[837]	validation-rmse:6.31100
[838]	validation-rmse:6.31083
[839]	validation-rmse:6.31074
[840]	validation-rmse:6.31056
[841]	validation-rmse:6.31045
[842]	validation-rmse:6.31045
[843]	validation-rmse:6.31039
[844]	validation-rmse:6.31026
[845]	validation-rmse:6.31024
[846]	validation-rmse:6.31018
[847]	validation-rmse:6.31003
[848]	validation-rmse:6.30992
[849]	validation-rmse:6.30993
[850]	validation-rmse:6.30999
[851]	validation-rmse:6.30970
[852]	validation-rmse:6.30967
[853]	validation-rmse:6.30965
[854]	vali

In [37]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run(experiment_id=experiment_id):

        mlflow.log_param("train-data-path", "../01-intro/data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "../01-intro/data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

KeyboardInterrupt: 